In [7]:
import pandas as pd
import numpy as np
import holidays
daily=pd.read_csv("day.csv")
hourly=pd.read_csv("hour.csv")
daily=daily.rename(columns={"dteday":"date","yr":"year","atemp":"ftemp","mnth":"month","weathersit":"weather"})
hourly=hourly.rename(columns={"dteday":"date","yr":"year","atemp":"ftemp","mnth":"month","hr":"hour","weathersit":"weather"})
daily["date"]=pd.to_datetime(daily["date"])
hourly["date"]=pd.to_datetime(hourly["date"])
#Weather data was sourced from the Integrated Surface Database (ISD) of the National Oceanic and Atmospheric Agency (NOAA) of the US
weather_daily=pd.read_csv("weather_daily.csv")
weather_daily["datetime"]=pd.to_datetime(weather_daily["datetime"])
weather_hourly=pd.read_csv("weather_hourly.csv")
weather_hourly["datetime"]=pd.to_datetime(weather_hourly["datetime"])
#No missing values
#Add the hour timestamp in the datetime column for easier comparison
for i in range(len(hourly)):
    hourly.loc[i,"date"]=hourly.loc[i,"date"].replace(hour=hourly.loc[i,"hour"])
#There are some hours missing in the hourly dataset. We will assign nan values for the missing hours
rang=pd.date_range(start=hourly.loc[0,"date"],end=hourly.loc[17378,"date"],freq="H")
hourly=hourly.set_index("date")
hourly=hourly.reindex(rang,fill_value=np.nan)
hourly=hourly.reset_index()
hourly=hourly.rename(columns={"index":"date"})
#We saw that the seasons column was totally scrambled, so we can not trust the other date columns to be correct
#We will extract the specifics from the original date again
daily["year"]=pd.DatetimeIndex(daily["date"]).year
daily["month"]=pd.DatetimeIndex(daily["date"]).month
daily["day"]=pd.DatetimeIndex(daily["date"]).day
daily["weekday"]=pd.DatetimeIndex(daily["date"]).weekday
hourly["year"]=pd.DatetimeIndex(hourly["date"]).year
hourly["month"]=pd.DatetimeIndex(hourly["date"]).month
hourly["day"]=pd.DatetimeIndex(hourly["date"]).day
hourly["hour"]=pd.DatetimeIndex(hourly["date"]).hour
hourly["weekday"]=pd.DatetimeIndex(hourly["date"]).weekday
#Do some renaming and categorization for some columns
daily["weekday"]=daily["weekday"].replace({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun"}).astype("category")
hourly["weekday"]=hourly["weekday"].replace({0:"Mon",1:"Tue",2:"Wed",3:"Thu",4:"Fri",5:"Sat",6:"Sun"}).astype("category")
daily=daily[["instant","date","year","month","day","season","weekday","workingday","holiday","weather","temp","ftemp","hum","windspeed","casual","registered","cnt"]]
hourly=hourly[["instant","date","year","month","day","hour","season","weekday","workingday","holiday","weather","temp","ftemp","hum","windspeed","casual","registered","cnt"]]

In [2]:
#Figures from the Metropolitan Washington Airports Authority for Reagan airport
reagan11=[1265192,1253227,1640947,1638472,1743876,1702388,1720092,1658578,1522587,1680103,1575307,1422325]
reagan12=[1292183,1310815,1649942,1683581,1724771,1754568,1795557,1854810,1606641,1726571,1700473,1555528]

In [8]:
#Check working days
for i in range(len(daily)):
    if daily.loc[i,"weekday"]=="Sat" or daily.loc[i,"weekday"]=="Sun": daily.loc[i,"workingday"]=0
    else: daily.loc[i,"workingday"]=1
for i in range(len(hourly)):
    if hourly.loc[i,"weekday"]=="Sat" or hourly.loc[i,"weekday"]=="Sun": hourly.loc[i,"workingday"]=0
    else: hourly.loc[i,"workingday"]=1
#Assign seasons based on the normal division of the year
for i in range(len(daily)):
    if daily.loc[i,"month"]<3 or daily.loc[i,"month"]==12: daily.loc[i,"season"]="Winter"
    elif daily.loc[i,"month"]<6: daily.loc[i,"season"]="Spring"
    elif daily.loc[i,"month"]<9: daily.loc[i,"season"]="Summer"
    else: daily.loc[i,"season"]="Autumn"
for i in range(len(hourly)):
    if hourly.loc[i,"month"]<3 or hourly.loc[i,"month"]==12: hourly.loc[i,"season"]="Winter"
    elif hourly.loc[i,"month"]<6: hourly.loc[i,"season"]="Spring"
    elif hourly.loc[i,"month"]<9: hourly.loc[i,"season"]="Summer"
    else: hourly.loc[i,"season"]="Autumn"

In [9]:
#Adding the weather data
daily["temp"]=weather_daily["temp"]
daily["ftemp"]=weather_daily["feelslike"]
daily["hum"]=weather_daily["humidity"]
daily["windspeed"]=weather_daily["windspeed"]
hourly["temp"]=weather_hourly["temp"]
hourly["ftemp"]=weather_hourly["feelslike"]
hourly["hum"]=weather_hourly["humidity"]
hourly["windspeed"]=weather_hourly["windspeed"]
#Daily weather
for i in range(len(weather_daily)):
    if weather_daily.loc[i,"cloudcover"]<34: daily.loc[i,"weather"]="good"
    elif weather_daily.loc[i,"cloudcover"]<67: daily.loc[i,"weather"]="cloudy"
    else: daily.loc[i,"weather"]="overcast"
for i in range(len(weather_daily)):
    if weather_daily.loc[i,"preciptype"]=="snow" or weather_daily.loc[i,"preciptype"]=="rain, snow": daily.loc[i,"weather"]="snow"
    elif weather_daily.loc[i,"preciptype"]=="rain": daily.loc[i,"weather"]="rain"
#Hourly weather
for i in range(len(weather_hourly)):
    if weather_hourly.loc[i,"cloudcover"]<34: hourly.loc[i,"weather"]="good"
    elif weather_hourly.loc[i,"cloudcover"]<67: hourly.loc[i,"weather"]="cloudy"
    else: hourly.loc[i,"weather"]="overcast"
for i in range(len(weather_hourly)):
    if weather_hourly.loc[i,"preciptype"]=="snow" or weather_hourly.loc[i,"preciptype"]=="rain, snow": hourly.loc[i,"weather"]="snow"
    elif weather_hourly.loc[i,"preciptype"]=="rain": hourly.loc[i,"weather"]="rain"
#Adding holiday dates    
dc_holidays=holidays.country_holidays("US",subdiv="DC")
for i in range(len(daily)):
    if daily.loc[i,"date"] in dc_holidays:
        daily.loc[i,"holiday"]=1
        daily.loc[i,"workingday"]=0
    else: 
        daily.loc[i,"holiday"]=0
for i in range(len(hourly)):
    if hourly.loc[i,"date"] in dc_holidays:
        hourly.loc[i,"holiday"]=1
        hourly.loc[i,"workingday"]=0
    else:
        hourly.loc[i,"holiday"]=0

In [5]:
#Nation's triathlon 11/09/2011
#Fancy food show 17-19/06/2011
#WH correspondents' 28/04/2011
#AIPAC and CPAC conferences 9-11/02 and 04-06/03
#Black caucus 21-24/09
#Campus pregress 26/03
#Cherry blosson 26/03-10/04


In [10]:
hourly.to_csv("hourly",encoding="utf-8",index=False)
daily.to_csv("daily",encoding="utf-8",index=False)